In [ ]:
import torch
import torch.nn as nn
from transformers import DistilBertModel
from test_harness import test_harness

class PhishingClassifier(nn.Module):
    def __init__(self):
        super(PhishingClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, html_input_ids, html_attention_mask):
        outputs = self.bert(input_ids=html_input_ids, attention_mask=html_attention_mask)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])
        return logits
    
    def test_name(self):
        return 'basic_html_transformer'

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model = PhishingClassifier().to(device)
test_harness(model, epochs=10)
torch.save(model.state_dict(), f"{model.test_name()}_phishing_classifier.pt")

